<a href="https://colab.research.google.com/github/s-kachroo/CMU-11785-TCR-CLEAN/blob/main/CMU_11785_CLEAN_Baseline_Step2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/s-kachroo/CMU-11785-TCR-CLEAN.git

Cloning into 'CMU-11785-TCR-CLEAN'...
remote: Enumerating objects: 24061, done.
remote: Counting objects: 100% (207/207), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 24061 (delta 159), reused 123 (delta 123), pack-reused 23854 (from 1)
Receiving objects: 100% (24061/24061), 172.12 MiB | 11.88 MiB/s, done.
Resolving deltas: 100% (244/244), done.
Updating files: 100% (23643/23643), done.


In [3]:
import os
os.chdir('/content/CMU-11785-TCR-CLEAN/app')
current_path = os.getcwd()
os.listdir(current_path)

['train-supconH.py',
 'results',
 'train-triplet.py',
 'requirements.txt',
 'CLEAN_infer_fasta.py',
 'DEMO.ipynb',
 'src',
 'build.py',
 'data',
 'inference.py',
 'gmm.py']

In [4]:
!pip install fair-esm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 3.8 MB/s eta 0:00:00


In [5]:
from src.CLEAN.utils import compute_esm_distance

train_csv = "filtered_train_df"
test_csv = "filtered_test_df"
val_csv = "filtered_val_df"
model_name = "esm2_t33_650M_1028_v1"


!cp -r /content/CMU-11785-TCR-CLEAN/app/data/1028/filtered_train_df.csv /content/CMU-11785-TCR-CLEAN/app/data/
!cp -r /content/CMU-11785-TCR-CLEAN/app/data/1028/filtered_test_df.csv /content/CMU-11785-TCR-CLEAN/app/data/
!cp -r /content/CMU-11785-TCR-CLEAN/app/data/1028/filtered_valid_df.csv /content/CMU-11785-TCR-CLEAN/app/data/


compute_esm_distance(train_csv, model_name)

100%|██████████| 192/192 [00:00<00:00, 2701.26it/s]


Calculating distance map, number of unique EC is 192


192it [00:00, 1580.88it/s]


In [6]:
import pickle
dist_map = pickle.load(open('/content/CMU-11785-TCR-CLEAN/app/data/distance_map/filtered_train_df.pkl', 'rb'))
print(dist_map)

{'FLKEKGGL': {'FLKEKGGL': 0.0, 'ILKEPVHGV': 0.7696163, 'EIYKRWII': 0.3666136, 'QVPLRPMTYK': 0.33803174, 'KRWIILGLNK': 0.26384047, 'KAFSPEVIPMF': 0.56136715, 'NEGVKAAW': 0.32420626, 'LQPFPQPELPYPQPQ': 0.61841255, 'LLQTGIHVRVSQPSL': 0.34570602, 'EHPTFTSQYRIQGKL': 0.37978178, 'PKYVKQNTLKLAT': 0.34225014, 'IVTDFSVIK': 0.2551365, 'ELAGIGILTV': 0.31719968, 'RAKFKQLL': 0.31200403, 'AVFDRKSDAK': 0.3209262, 'GILGFVFTL': 0.35420448, 'SLFNTVATLY': 0.52709246, 'RLRAEAQVK': 0.22028887, 'AYAQKIFKI': 0.25724432, 'LLDFVRFMGV': 0.4310071, 'KLGGALQAK': 0.2674622, 'FLYALALLL': 0.8205657, 'FLASKIGRLV': 0.47366625, 'GLCTLVAML': 0.31453624, 'IPSINVHHY': 0.35952005, 'LLFGYPVYV': 0.24844278, 'QYDPVAALF': 0.44567734, 'RTLNAWVKV': 0.3551128, 'KTWGQYWQV': 0.40721977, 'FLRGRAYGL': 0.43648452, 'TPRVTGGGAM': 0.29344818, 'RPPIFIRRL': 0.36472735, 'VTEHDTLLY': 0.20830403, 'QASQEVKNW': 0.36956528, 'TPQDLNTML': 0.40239874, 'YSEHPTFTSQY': 0.33226866, 'HPVGEADYFEY': 0.48019072, 'KRWIIMGLNK': 0.34535822, 'NLVPMVATV': 0.233

In [7]:
!git pull origin main

From https://github.com/s-kachroo/CMU-11785-TCR-CLEAN
 * branch              main       -> FETCH_HEAD
Already up to date.


In [8]:
!python train-triplet.py --training_data filtered_train_df --model_name esm2_t33_650M_1028_v1 --dir_name esm2_t33_650M_1028_v1 --epoch 20

==> device used: cuda:0 | dtype used:  torch.float32 
==> args: Namespace(learning_rate=0.0005, epoch=20, model_name='esm2_t33_650M_1028_v1', training_data='filtered_train_df', hidden_dim=512, out_dim=128, dir_name='esm2_t33_650M_1028_v1', adaptive_rate=100, verbose=False)
Mining hard negatives:
100% 192/192 [00:00<00:00, 5569.28it/s]
The number of unique EC numbers:  192
---------------------------------------------------------------------------
| end of epoch   1 | time:  1.73s | training loss 0.9818
---------------------------------------------------------------------------
---------------------------------------------------------------------------
| end of epoch   2 | time:  0.15s | training loss 0.9749
---------------------------------------------------------------------------
---------------------------------------------------------------------------
| end of epoch   3 | time:  0.14s | training loss 1.0039
--------------------------------------------------------------------------

In [9]:
from src.CLEAN.infer import infer_maxsep
model_name = 'esm2_t33_650M_1028_v1'
embedding_folder_name = 'esm2_t33_650M_1028_v1'
infer_maxsep(train_data=train_csv, test_data=test_csv, report_metrics=True, pretrained=False, model_name=model_name)

The embedding sizes for train and test: torch.Size([16532, 128]) torch.Size([4680, 128])


100%|██████████| 192/192 [00:00<00:00, 16236.34it/s]


Calculating eval distance map, between 4680 test ids and 192 train EC cluster centers


4680it [00:00, 5858.41it/s]


############ EC calling results using maximum separation ############
---------------------------------------------------------------------------
>>> total samples: 4680 | total ec: 191 
>>> precision: 0.014 | recall: 0.0205| F1: 0.0125 | AUC: 0.509 
---------------------------------------------------------------------------
